In [1]:
import pandas as pd
import numpy as np
import openpyxl
from pymongo import MongoClient
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi


In [2]:
# Open the xlsx file and convert to a dataframe
# Load spreadsheet
xl = pd.ExcelFile('MES.xlsx')
# Load a sheet into a DataFrame by its name
dfs = {sheet_name: xl.parse(sheet_name) for sheet_name in xl.sheet_names}

In [3]:
# Separate every element of the dictionary into a different dataframe
principales_cuentas = dfs['1']
balance_general = dfs['2']
estado_resultados = dfs['3']
ingresos_egresos = dfs['4']
detalle_ingresos = dfs['5']
detalle_egresos = dfs['6']
indicadores = dfs['7']

In [4]:
principales_cuentas_columns = principales_cuentas.columns.tolist()
balance_general_columns = balance_general.columns.tolist()
estado_resultados_columns = estado_resultados.columns.tolist()
ingresos_egresos_columns = ingresos_egresos.columns.tolist()
detalle_ingresos_columns = detalle_ingresos.columns.tolist()
detalle_egresos_columns = detalle_egresos.columns.tolist()
indicadores_columns = indicadores.columns.tolist()

In [18]:
rows = ["Total de Activos", "Total de Pasivos", "Capital Social", "Resultado del Ejercicio", "Total Patrimonio Neto", "Primas Directas", "Primas Reaseguros Aceptados - Local", 
        "Siniestros Seguros Directos", "Resultado Técnico Bruto [7]=[3]-[6]", "Gastos De Producción", "Gastos De Cesión Reaseguros - Local", "Gastos De Cesión Reaseguros - Exterior", 
        "Gastos Técnicos De Explotación", "Constitución De Previsiones", "Resultado Técnico Neto [11]=[7]+[10]", "Resultado Total del Ejercicio", "Resultado del Ejercicio"]
columns = ['El Comercio Paraguayo S.A. De Seguros', 'La Rural S.A. De Seguros', 'La Paraguaya S.A. De Seguros', 'Seguros Generales S. A. (Segesa)', 'Rumbos S.A. De Seguros', 
           'La Consolidada S.A. De Seguros', 'El Productor S.A. De Seguros Y Reaseguros', 'Atalaya S.A De Seguros Generales', 'La Independencia De Seguros Sociedad Anonima', 
           'Patria S.A. De Seguros Y Reaseguros', 'Alianza Garantía Seguros Y Reaseguros S.A.', 'Aseguradora Paraguaya S.A', 'Fénix S.A. De Seguros Y Reaseguros', 
           'Central S.A. De Seguros', 'Seguros Chaco S.A. De Seguros Y Reaseguros', 'El Sol Del Paraguay Compañía De Seguros Y Reaseguros', 'Intercontinental De Seguros Y Reaseguros S.A.', 
           'Seguridad S.A. Compañía De Seguros', 'Aseguradora Yacyreta S.A. De Seguros Y Reaseguros', 'La Agrícola S.A. De Seguros Y Reaseguros', 'Ueno Seguros S.A.', 'Cenit De Seguros S.A.', 
           'La Meridional Paraguaya S.A. De Seguros', 'Aseguradora Del Este S.A De Seguros Y Reaseguros', 'Regional S.A. De Seguros Y Reaseguros', 'Mapfre Paraguay Compañía De Seguros S.A.', 
           'Aseguradora Tajy Propiedad Cooperativa S.A. De Seguros', 'Panal Compañía De Seguros Generales S.A.', 'Sancor Seguros Del Paraguay S.A.', 'Royal Seguros S.A. Compañía De Seguros', 
           'Nobleza Seguros S.A. Compañia De Seguros', 'Itau Seguros Paraguay S.A.', 'Familiar Seguros S.A.', 'Atlas S.A. De Seguros', 'Total Mercado']
dataframes = [principales_cuentas, balance_general, estado_resultados, ingresos_egresos, detalle_ingresos, detalle_egresos, indicadores]
dataframes_names = ["principales_cuentas", "balance_general", "estado_resultados", "ingresos_egresos", "detalle_ingresos", "detalle_egresos", "indicadores"]
print(columns)

['El Comercio Paraguayo S.A. De Seguros', 'La Rural S.A. De Seguros', 'La Paraguaya S.A. De Seguros', 'Seguros Generales S. A. (Segesa)', 'Rumbos S.A. De Seguros', 'La Consolidada S.A. De Seguros', 'El Productor S.A. De Seguros Y Reaseguros', 'Atalaya S.A De Seguros Generales', 'La Independencia De Seguros Sociedad Anonima', 'Patria S.A. De Seguros Y Reaseguros', 'Alianza Garantía Seguros Y Reaseguros S.A.', 'Aseguradora Paraguaya S.A', 'Fénix S.A. De Seguros Y Reaseguros', 'Central S.A. De Seguros', 'Seguros Chaco S.A. De Seguros Y Reaseguros', 'El Sol Del Paraguay Compañía De Seguros Y Reaseguros', 'Intercontinental De Seguros Y Reaseguros S.A.', 'Seguridad S.A. Compañía De Seguros', 'Aseguradora Yacyreta S.A. De Seguros Y Reaseguros', 'La Agrícola S.A. De Seguros Y Reaseguros', 'Ueno Seguros S.A.', 'Cenit De Seguros S.A.', 'La Meridional Paraguaya S.A. De Seguros', 'Aseguradora Del Este S.A De Seguros Y Reaseguros', 'Regional S.A. De Seguros Y Reaseguros', 'Mapfre Paraguay Compañía 

In [6]:
all_equal = (balance_general_columns == estado_resultados_columns == 
             ingresos_egresos_columns == detalle_ingresos_columns == detalle_egresos_columns == indicadores_columns)

print("All column lists are equal:" , all_equal)

All column lists are equal: True


In [7]:
def get_value_from_df(df, row_value, column_name):
    """
    Retrieves the value from a DataFrame based on the given row value and column name.

    Parameters:
    df (pandas.DataFrame): The DataFrame to search in.
    row_value (str): The value to search for in the 'Nombre de la Cuenta' column.
    column_name (str): The name of the column to retrieve the value from.

    Returns:
    str or Error message: The value at the intersection of the row and column_name, or an error message if the row or column names are invalid.
    """
    try:
        # Find the row where 'Nombre de la Cuenta' is equal to row_value
        row = df[df['Nombre de la Cuenta'] == row_value]
        
        # Return the value at the intersection of the row and column_name
        return row[column_name].values[0]
    except (IndexError, KeyError):
        return "Error: Invalid row or column names. Please verify and try again."

In [8]:
import pandas as pd

def create_new_df(dfs, rows, columns):
    """
    Create a new dataframe by extracting values from multiple dataframes.

    Args:
        dfs (list): A list of dataframes from which values will be extracted.
        rows (list): A list of row names.
        columns (list): A list of column names.

    Returns:
        pandas.DataFrame: A new dataframe with extracted values.

    """
    # Create a new dataframe with the first column as "Nombre de la Cuenta"
    new_df = pd.DataFrame(rows, columns=["Nombre de la Cuenta"])

    # Add new columns to the dataframe
    for column in columns:
        new_df[column] = None

    # Iterate over each dataframe
    for df in dfs:
        # Iterate over each row
        for i, row in enumerate(rows):
            # Iterate over each column
            for column in columns:
                # Get the value at the intersection of the row and column
                value = get_value_from_df(df, row, column)

                # If the value is not an error message, assign it to the new dataframe
                if not isinstance(value, str):
                    new_df.at[i, column] = value

    # Return the new dataframe
    return new_df

In [9]:
def create_new_dfs(dfs,insurer, rows, columns):
    # Create a new dataframe with the first column as "Nombre de la Cuenta"
    new_df = pd.DataFrame(rows, columns=["Nombre de la Cuenta"])

    # Add new columns to the dataframe

    new_df[insurer] = None

    # Iterate over each dataframe
    for df in dfs:
        # Iterate over each row
        for i, row in enumerate(rows):
            # Iterate over each column
            for column in columns:
                # Get the value at the intersection of the row and column
                value = get_value_from_df(df, row, column)

                # If the value is not an error message, assign it to the new dataframe
                if not isinstance(value, str):
                    new_df.at[i, insurer] = value

    # Return the new dataframe
    return new_df

In [10]:
def create_dict_from_df(df):
    # Initialize an empty dictionary
    new_dict = {}

    # Iterate over each row in the dataframe
    for _, row in df.iterrows():
        # Use the 'Nombre de la Cuenta' value as the key and the 'El Comercio Paraguayo S.A. De Seguros' value as the value
        new_dict[row['Nombre de la Cuenta']] = row['El Comercio Paraguayo S.A. De Seguros']

    # Return the new dictionary
    return new_dict

In [11]:
def create_dict_from_df2(df):
    # Initialize an empty dictionary
    new_dict = {}

    # Get the second key from the old dictionary
    second_key = list(df.columns)[1]

    # Set the first key-value pair
    new_dict["Insurer"] = second_key

    # Iterate over each row in the dataframe
    for _, row in df.iterrows():
        # Use the 'Nombre de la Cuenta' value as the key and the 'El Comercio Paraguayo S.A. De Seguros' value as the value
        new_dict[row['Nombre de la Cuenta']] = row[second_key]

    # Return the new dictionary
    return new_dict

In [ ]:
def transform_keys(input_dict):
    # Define a mapping from the old keys to the new keys
    key_mapping = {
        "Total de Activos": "total_activos",
        "Total de Pasivos": "total_pasivos",
        "Capital Social": "capital_social",
        "Resultado del Ejercicio": "resultado_ejercicio",
        "Total Patrimonio Neto": "total_patrimonio_neto",
        "Primas Directas": "primas_directas",
        "Primas Reaseguros Aceptados - Local": "primas_reaseguros_aceptados_local",
        "Siniestros Seguros Directos": "siniestros_seguros_directos",
        "Resultado Técnico Bruto [7]=[3]-[6]": "resultado_tecnico_bruto",
        "Gastos De Producción": "gastos_produccion",
        "Gastos De Cesión Reaseguros - Local": "gastos_cesion_reaseguros_local",
        "Gastos De Cesión Reaseguros - Exterior": "gastos_cesion_reaseguros_exterior",
        "Gastos Técnicos De Explotación": "gastos_tecnicos_explotacion",
        "Constitución De Previsiones": "constitucion_previsiones",
        "Resultado Técnico Neto [11]=[7]+[10]": "resultado_tecnico_neto",
        "Resultado Total del Ejercicio": "resultado_total_ejercicio",
        "Resultado del Ejercicio": "resultado_ejercicio"
    }

    # Create a new dictionary with the transformed keys
    output_dict = {key_mapping[key]: value for key, value in input_dict.items() if key in key_mapping}

    return output_dict

In [17]:
container = create_new_dfs(dataframes,'El Comercio Paraguayo S.A. De Seguros', rows, columns)
print(dataframes)
#conver df to excel
container.to_excel("output.xlsx")

[                                             N° Cuenta  \
0                                      Balance General   
1                                       01.01.00.00.00   
2                                       01.02.00.00.00   
3                                       01.03.00.00.00   
4                                       01.04.00.00.00   
..                                                 ...   
116                                                NaN   
117                                                NaN   
118                                                NaN   
119                                                NaN   
120  *Obs.: Las celdas en donde se dividen numeros ...   

            Nombre de la Cuenta     2013-2014     2014-2015     2015-2016  \
0                           NaN           NaN           NaN           NaN   
1              Disponibilidades  2.206323e+11  2.492709e+11  2.598638e+11   
2    Créditos Técnicos Vigentes  7.009540e+11  7.444147e+11  8.540235e+

In [13]:
print(container.keys()) 
print(container)

Index(['Nombre de la Cuenta', 'El Comercio Paraguayo S.A. De Seguros'], dtype='object')
                       Nombre de la Cuenta  \
0                         Total de Activos   
1                         Total de Pasivos   
2                           Capital Social   
3                  Resultado del Ejercicio   
4                    Total Patrimonio Neto   
5                          Primas Directas   
6      Primas Reaseguros Aceptados - Local   
7              Siniestros Seguros Directos   
8      Resultado Técnico Bruto [7]=[3]-[6]   
9                     Gastos De Producción   
10     Gastos De Cesión Reaseguros - Local   
11  Gastos De Cesión Reaseguros - Exterior   
12          Gastos Técnicos De Explotación   
13             Constitución De Previsiones   
14    Resultado Técnico Neto [11]=[7]+[10]   
15           Resultado Total del Ejercicio   
16                 Resultado del Ejercicio   

   El Comercio Paraguayo S.A. De Seguros  
0                          5907591487366

In [14]:
new_dict = create_dict_from_df2(container)
print(new_dict.keys())
print(type(new_dict))
print(new_dict)


dict_keys(['Insurer', 'Total de Activos', 'Total de Pasivos', 'Capital Social', 'Resultado del Ejercicio', 'Total Patrimonio Neto', 'Primas Directas', 'Primas Reaseguros Aceptados - Local', 'Siniestros Seguros Directos', 'Resultado Técnico Bruto [7]=[3]-[6]', 'Gastos De Producción', 'Gastos De Cesión Reaseguros - Local', 'Gastos De Cesión Reaseguros - Exterior', 'Gastos Técnicos De Explotación', 'Constitución De Previsiones', 'Resultado Técnico Neto [11]=[7]+[10]', 'Resultado Total del Ejercicio'])
<class 'dict'>
{'Insurer': 'El Comercio Paraguayo S.A. De Seguros', 'Total de Activos': 5907591487366, 'Total de Pasivos': 3458736792972, 'Capital Social': 1444211080763, 'Resultado del Ejercicio': 244190177799, 'Total Patrimonio Neto': 2448854694394, 'Primas Directas': 2218164421033.0, 'Primas Reaseguros Aceptados - Local': 15565099667, 'Siniestros Seguros Directos': 778719598639, 'Resultado Técnico Bruto [7]=[3]-[6]': 963020974727, 'Gastos De Producción': 431669724235, 'Gastos De Cesión Re

In [15]:
def insert_data(data_dict, collection_name):
    uri = "MONGODB_URI"
    # Create a new client and connect to the server
    client = MongoClient(uri, server_api=ServerApi('1'))    

    # Access the 'insurance_market_analisis' database
    db = client['insurance_market_statistics']

    # Convert numpy.int64 to int
    for key, value in data_dict.items():
        if isinstance(value, np.int64):
            data_dict[key] = int(value)

    # Access the specified collection
    collection = db[collection_name]

    # Insert the data
    collection.insert_one(data_dict)

In [16]:
for insurer in columns:
    container = create_new_dfs(dataframes, insurer, rows, columns)
    new_dict = create_dict_from_df2(container)
    insert_data(new_dict, "insurance_companies")
    print(new_dict)
    

ServerSelectionTimeoutError: mongodb_uri:27017: [Errno 11001] getaddrinfo failed (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 65f9b22509bb5300d98585d0, topology_type: Unknown, servers: [<ServerDescription ('mongodb_uri', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('mongodb_uri:27017: [Errno 11001] getaddrinfo failed (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>